In [31]:
import os
import numpy as np
from pathlib import Path

import pandas as pd
import sys
# kouchou-aiのパスを追加
sys.path.append("../kouchou-ai/server")
sys.path.append("../kouchou-ai/server/broadlistening/pipeline/")

In [32]:
slug = "rep-election-2"
exp_prefix = f"20250415_プロンプト修正_{slug}"
DATA_DIR = Path("../data/original_outputs")# 広聴AIで出力したデータを保存しているディレクトリ
OUTPUT_DIR = Path("../data/experiment_outputs")

# クラスタリング

In [33]:
from umap import UMAP
from broadlistening.pipeline.steps.hierarchical_clustering import hierarchical_clustering_embeddings

arguments_df = pd.read_csv(DATA_DIR / slug /  "args.csv")
embeddings_df = pd.read_pickle(DATA_DIR / slug / "embeddings.pkl")
embeddings_array = np.asarray(embeddings_df["embedding"].values.tolist())
cluster_nums = [15, 70]
n_samples = embeddings_array.shape[0]
default_n_neighbors = 15

if n_samples <= default_n_neighbors:
    n_neighbors = max(2, n_samples - 1)
else:
    n_neighbors = default_n_neighbors

umap_model = UMAP(random_state=42, n_components=2, n_neighbors=n_neighbors)
umap_embeds = umap_model.fit_transform(embeddings_array)


/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [34]:
# 広聴AIで出力したレポートのslug
cluster_results = hierarchical_clustering_embeddings(
    umap_embeds=umap_embeds,
    cluster_nums=cluster_nums,
)
result_df = pd.DataFrame(
    {
        "arg-id": arguments_df["arg-id"],
        "argument": arguments_df["argument"],
        "x": umap_embeds[:, 0],
        "y": umap_embeds[:, 1],
    }
)
for cluster_level, final_labels in enumerate(cluster_results.values(), start=1):
    result_df[f"cluster-level-{cluster_level}-id"] = [f"{cluster_level}_{label}" for label in final_labels]



start initial clustering
end initial clustering
start hierarchical clustering
[15, 70]
n_cluster_cut:  15
end hierarchical clustering


In [35]:
clusters_argument_df =  result_df.copy()
orig_initial_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_initial_labels.csv")
orig_merge_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_merge_labels.csv")

exp_output_dir = OUTPUT_DIR / exp_prefix

os.makedirs(exp_output_dir, exist_ok=True)


# プロンプト修正の実験

## 初期ラベリング

In [36]:
INITIAL_PROMPT = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説し、表札（label）をつけてください。
表札については、グループ内の具体的な論点や特徴を反映した、具体性の高い名称を考案してください。  
出力はJSONとし、フォーマットは以下のサンプルを参考にしてください。


# サンプルの入出力
## 入力例
- 手作業での意見分析は時間がかかりすぎる。AIで効率化できると嬉しい
- 今のやり方だと分析に工数がかかりすぎるけど、AIならコストをかけずに分析できそう
- AIが自動で意見を整理してくれると楽になって嬉しい


## 出力例
{{
    "label": "AIによる業務効率の大幅向上とコスト効率化",
    "description": "このクラスタは、従来の手作業による意見分析と比較して、AIによる自動化で分析プロセスが効率化され、作業時間の短縮や運用コストの効率化が実現される点に対する前向きな評価が中心です。"
}}"""

In [37]:
from broadlistening.pipeline.steps.hierarchical_initial_labelling import initial_labelling

In [38]:
cluster_id_columns = [col for col in clusters_argument_df.columns if col.startswith("cluster-level-")]
initial_cluster_id_column = cluster_id_columns[-1]
sampling_num = 30
initial_labelling_prompt = INITIAL_PROMPT
model = "gpt-4o"
workers = 70

initial_label_df = initial_labelling(
    initial_labelling_prompt,
    clusters_argument_df,
    sampling_num,
    model,
    workers,
)

In [39]:
# 最下層のクラスタ。プロンプト修正後
initial_label_df.head()

,cluster_id,label,description
0,2_42,選挙結果に対する多様な反応と期待,このクラスタは、選挙結果に対する有権者の多様な反応や期待を反映しています。具体的には、当選や...
1,2_16,立憲民主党の地域別支持動向と選挙結果の驚き,このクラスタは、立憲民主党が特定の地域で強い支持を得ていることや、選挙結果に対する驚きの声に...
2,2_61,選挙に対する多様な関心と期待の高まり,このクラスタは、選挙に対する多様な関心や期待が高まっていることを示しています。選挙が祭りのよ...
3,2_20,低投票率に対する懸念とその影響に関する多角的視点,このクラスタは、選挙における低投票率に対する驚きや懸念の声を中心に、投票率の低さが日本の未来...
4,2_5,2024年衆院選に対する多様な反応と関心の高まり,このクラスタは、2024年の衆議院選挙に対する多様な反応や意見を集めたものです。選挙結果に対...


In [40]:
initial_clusters_argument_df = clusters_argument_df.merge(
    initial_label_df,
    left_on=initial_cluster_id_column,
    right_on="cluster_id",
    how="left",
).rename(
    columns={
        "label": f"{initial_cluster_id_column.replace('-id', '')}-label",
        "description": f"{initial_cluster_id_column.replace('-id', '')}-description",
    }
)

In [41]:
initial_clusters_argument_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description
0,Acsv-1_0,当選確実の報道がある一方で、落選確実の状況もあるという意見がある。,9.395892,4.318309,1_15,2_42,2_42,選挙結果に対する多様な反応と期待,このクラスタは、選挙結果に対する有権者の多様な反応や期待を反映しています。具体的には、当選や...
1,Acsv-2_0,愛媛で自民党が1区と3区、立憲民主党が2区で当選したという意見があるが、政権交代がどう影響す...,6.425354,2.939700,1_9,2_16,2_16,立憲民主党の地域別支持動向と選挙結果の驚き,このクラスタは、立憲民主党が特定の地域で強い支持を得ていることや、選挙結果に対する驚きの声に...
2,Acsv-3_0,選挙の結果が気になって食事に行けないという声がある。,11.113695,5.199418,1_14,2_61,2_61,選挙に対する多様な関心と期待の高まり,このクラスタは、選挙に対する多様な関心や期待が高まっていることを示しています。選挙が祭りのよ...
3,Acsv-4_0,選挙に行かなかった人に対して消費税を20%にするという意見がある。,9.655325,7.321733,1_13,2_20,2_20,低投票率に対する懸念とその影響に関する多角的視点,このクラスタは、選挙における低投票率に対する驚きや懸念の声を中心に、投票率の低さが日本の未来...
4,Acsv-5_0,衆院選が始まり、選挙戦が面白くなってきたという意見がある。,9.691563,4.819920,1_14,2_5,2_5,2024年衆院選に対する多様な反応と関心の高まり,このクラスタは、2024年の衆議院選挙に対する多様な反応や意見を集めたものです。選挙結果に対...
...,...,...,...,...,...,...,...,...,...
10129,Acsv-10085_0,自民党が議席を大幅に減らす可能性があるという意見がある。,3.502974,5.337620,1_8,2_55,2_55,自民党の議席減少と選挙結果に対する多様な反応,このクラスタは、衆院選における自民党の議席減少に関する様々な意見や反応を集めたものです。自民...
10130,Acsv-10086_0,選挙速報を楽しんで見ているという意見がある。,12.083847,4.712981,1_12,2_40,2_40,選挙速報の視聴体験とその多様な感情,このクラスタは、選挙速報を視聴することに対する多様な感情や行動に焦点を当てています。選挙速報...
10131,Acsv-10089_0,死刑囚が選挙権を持たないことに対する意見が述べられている。,10.587885,6.653868,1_13,2_0,2_0,選挙の重要性と投票率向上への多角的アプローチ,このクラスタは、選挙の重要性を強調し、投票率を向上させるための様々な意見や提案が集まっていま...
10132,Acsv-10090_0,麻生太郎氏が衆院選で当選したとの情報がある。,6.751141,1.018640,1_11,2_24,2_24,選挙結果に対する不信感と政治家への批判的意見,このクラスタは、選挙結果に対する不信感や、特定の政治家の当選に対する批判的な意見が中心です。...


In [42]:
initial_clusters_argument_df.to_csv(exp_output_dir / "hierarchical_initial_labels.csv", index=False)

## 統合ラベリング

In [43]:
from broadlistening.pipeline.steps.hierarchical_merge_labelling import _filter_id_columns, merge_labelling, melt_cluster_data, _build_parent_child_mapping, calculate_cluster_density

In [44]:
MERGE_PROMPT = """あなたはデータ分析のエキスパートです。
現在、テキストデータの階層クラスタリングを行っています。
下層のクラスタのタイトルと説明、およびそれらのクラスタが所属する上層のクラスタのテキストのサンプルを与えるので、上層のクラスタのタイトルと説明を作成してください。

# 指示
- 統合後のクラスタ名は、統合前のクラスタ名称をそのまま引用せず、内容に基づいた新たな名称にしてください。  
- タイトルには、具体的な事象・行動（例：地域ごとの迅速対応、復興計画の着実な進展、効果的な情報共有・地域協力など）を含めてください
  - 可能な限り具体的な表現を用いるようにし、抽象的な表現は避けてください
    - 「多様な意見」などの抽象的な表現は避けてください
- 出力例に示したJSON形式で出力してください


# サンプルの入出力
## 入力例
- 「顧客フィードバックの自動集約」: このクラスタは、SNSやオンラインレビューなどから集めた大量の意見をAIが瞬時に解析し、企業が市場のトレンドや顧客の要望を即時に把握できる点についての期待を示しています。
- 「AIによる業務効率の大幅向上とコスト効率化」: このクラスタは、従来の手作業による意見分析と比較して、AIによる自動化で分析プロセスが効率化され、作業時間の短縮や運用コストの効率化が実現される点に対する前向きな評価が中心です。

## 出力例
{{
    "label": "AI技術の導入による意見分析の効率化への期待",
    "description": "大量の意見やフィードバックから迅速に洞察を抽出できるため、企業や自治体が消費者や市民の声を的確に把握し、戦略的な意思決定やサービス改善が可能になります。また、従来の手法と比べて作業負荷が軽減され、業務効率の向上やコスト削減といった実際の便益が得られると期待されています。"
}}"""

In [45]:
config = {}
config["hierarchical_merge_labelling"] = {
    "prompt": MERGE_PROMPT,
    "model": "gpt-4o",
    "sampling_num": 50,
    "workers": 70,
}

In [46]:
clusters_df = initial_clusters_argument_df.copy()
cluster_id_columns: list[str] = _filter_id_columns(clusters_df.columns)
merge_result_df = merge_labelling(
    clusters_df=clusters_df,
    cluster_id_columns=sorted(cluster_id_columns, reverse=True),
    config=config,
)

100%|██████████| 1/1 [00:13<00:00, 13.41s/it]


In [47]:
melted_df = melt_cluster_data(merge_result_df)
parent_child_df = _build_parent_child_mapping(merge_result_df, cluster_id_columns)
melted_df = melted_df.merge(parent_child_df, on=["level", "id"], how="left")
melted_df.head(20)

,level,id,label,description,value,parent
0,1,1_15,選挙結果に対する国民の多様な反応と政治不信の表出,選挙結果に対する国民の反応は多岐にわたり、当選や落選の速報に対する驚きや期待、特定の候補者に...,691,0
1,1,1_9,地域ごとの選挙結果と支持動向の多様性に対する驚きと分析,日本各地での選挙結果に対する有権者の反応や分析が多様であることが示されています。特定の地域で...,708,0
2,1,1_14,選挙報道と政治参加意識の高まりによる社会的関心の拡大,選挙に対する多様な関心が高まり、選挙報道や結果に対する期待、不安、驚きが交錯しています。Vt...,699,0
3,1,1_13,選挙参加促進と制度改善への具体的提案と懸念,選挙における低投票率の影響や、選挙制度に対する不信感が広がる中、投票率向上のための具体的な提...,1299,0
4,1,1_4,選挙における政党の議席動向と略称問題への対応,選挙において、立憲民主党や国民民主党、参政党、日本保守党などの議席動向が注目されています。特...,681,0
5,1,1_10,選挙結果に対する多様な感情と評価の表出,選挙結果に対する有権者の多様な感情や評価が集まっており、特定の候補者の当選に対する驚きや喜び...,723,0
6,1,1_7,選挙結果に対する多様な反応と政治的課題の浮き彫り,自民党を中心とした選挙結果に対する国民の多様な反応が示されており、特に裏金問題や報道の公平性...,603,0
7,1,1_12,選挙報道とエンターテインメントの同時進行における視聴者の多様な反応とメディアの役割,選挙特番と日本シリーズの同時放送に対する視聴者の反応は多岐にわたります。視聴者は、選挙報道の...,807,0
8,1,1_11,選挙結果に対する不確定情報と政治的変化への期待,選挙結果に関する不確定情報や虚偽の可能性が指摘される中、特定の政治家の当選や落選に対する多様...,692,0
9,1,1_3,日本の政治情勢における連立政権の変動と新勢力の台頭,自民党と公明党の連立政権が次の衆院選で過半数を維持できるかどうかに対する不安が広がる中、選挙...,973,0


In [48]:
melted_df.to_csv(exp_output_dir / "プロンプト変更後のラベリング結果.csv", index=False)

In [49]:
# 自動評価

In [50]:
import json
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
random.seed(42)

from broadlistening.pipeline.services.llm import request_to_chat_openai

In [51]:
EVALUATION_PROMPT = """あなたはテキストクラスタリングの専門家です。クラスタリング結果のタイトルの適切性を評価してください。

# 指示
与えられた情報に基づいて、クラスタのタイトルがそのクラスタに含まれるテキストデータをどれだけ適切に表現しているかを評価してください。
評価結果は出力例に記載したjson形式で出力してください。

# 入力情報
1. 評価対象クラスタのタイトル
2. 評価対象クラスタに所属するテキストデータのサンプル
3. 他のクラスタからサンプリングしたテキストデータ

# 評価基準
以下の観点から、クラスタのタイトルの適切性を1〜5の5段階で評価してください：

1. inclusiveness_score: タイトルがクラスタ内のすべてのテキストの共通テーマを捉えているか（1: 全く捉えていない 〜 5: 完全に捉えている）
2. specificity_score: タイトルがこのクラスタを他のクラスタと明確に区別できるか（1: 全く区別できない 〜 5: 完全に区別できる）
3. concreteness_score: タイトルが抽象的すぎず、具体的な内容を示しているか（1: 非常に抽象的 〜 5: 非常に具体的）

# 評価例

## 入力例
評価対象クラスタのタイトル: 「災害対応の迅速化と情報共有の強化」

評価対象クラスタに所属するテキストデータのサンプル:
- 災害発生時の初動対応をもっと迅速にしてほしい。特に高齢者への支援が遅れがちだ。
- 避難所の情報がリアルタイムで更新されず、どこに行けばいいのか混乱した。
- 災害時の情報共有システムを強化し、住民が必要な情報にすぐアクセスできるようにすべき。
- 地域ごとの災害対策本部の連携が不十分で、支援物資の配布に無駄が生じていた。
- 災害発生から避難指示が出るまでのタイムラグを短縮する必要がある。

他のクラスタからサンプリングしたテキストデータ:
- 復興予算の使い道をもっと透明化してほしい。どこにいくら使われているのか不明確だ。
- 被災地域の経済復興のための長期的な計画が見えてこない。
- 防災教育を学校カリキュラムに積極的に取り入れるべきだ。
- ボランティアの受け入れ体制が整っておらず、せっかくの支援の手が活かされていない。

## 出力例
{{
  "inclusiveness_score": {{
      "score": 4,
      "reason": "タイトル「災害対応の迅速化と情報共有の強化」は、クラスタ内のテキストの主要なテーマである「初動対応の迅速化」「情報共有の問題」「対策本部の連携」などの要素を広くカバーしています。ただし、「高齢者への支援」という具体的な対象については明示されていないため、完全ではありません。"
  }},
  "specificity_score": {{
      "score": 4,
      "reason": "このタイトルは「対応の迅速化」と「情報共有」という2つの明確な焦点を持っており、他のクラスタ（予算の透明化、経済復興、防災教育など）とは明確に区別できます。ただし、災害対応の中でも「初動」に関する内容が多いため、もう少し特化した表現があるとより区別しやすくなります。"
  }},
  "concreteness_score": {{
      "score": 3,
      "reason": "「迅速化」と「情報共有の強化」という方向性は示されていますが、どのような迅速化（初動対応、避難指示など）なのか、どのような情報共有（避難所情報、支援物資など）なのかまでは具体化されていません。"
  }},
}}
"""

In [52]:
def sample_cluster_data(
    df: pd.DataFrame,
    cluster_column: str,
    cluster_id: str,
    text_column: str,
    sample_size: int
) -> list[str]:
    """クラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        cluster_id: サンプリング対象のクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: サンプリングするデータ数

    Returns:
        サンプリングしたテキストデータのリスト
    """
    cluster_data = df[df[cluster_column] == cluster_id]
    sample_size = min(sample_size, len(cluster_data))
    
    if sample_size == 0:
        return []
    
    return cluster_data.sample(sample_size)[text_column].tolist()


def sample_other_clusters_data(
    df: pd.DataFrame,
    cluster_column: str,
    exclude_cluster_id: str,
    text_column: str,
    sample_size: int,
    sample_clusters: int | None = None
) -> list[str]:
    """他のクラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        exclude_cluster_id: 除外するクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: 各クラスタからサンプリングするデータ数
        sample_clusters: サンプリングするクラスタ数（Noneの場合は全クラスタ）

    Returns:
        サンプリングしたテキストデータのリスト
    """
    other_clusters = df[df[cluster_column] != exclude_cluster_id][cluster_column].unique()
    
    if len(other_clusters) == 0:
        return []
    
    if sample_clusters is not None and sample_clusters < len(other_clusters):
        other_clusters = random.sample(list(other_clusters), sample_clusters)
    
    sampled_data = []
    for cluster_id in other_clusters:
        cluster_samples = sample_cluster_data(
            df, cluster_column, cluster_id, text_column, sample_size
        )
        sampled_data.extend(cluster_samples)
    
    return sampled_data


def evaluate_cluster_title(
    prompt: str,
    cluster_title: str,
    cluster_samples: list[str],
    other_clusters_samples: list[str],
    model: str = "gpt-4o"
) -> dict[str, dict]:
    """クラスタのタイトルを評価する

    Args:
        prompt: 評価用プロンプト
        cluster_title: 評価対象のクラスタタイトル
        cluster_samples: 評価対象クラスタのサンプルデータ
        other_clusters_samples: 他のクラスタのサンプルデータ
        model: 使用するLLMモデル名

    Returns:
        評価結果（JSON形式）
    """
    # 入力データの整形
    input_text = f"評価対象クラスタのタイトル: 「{cluster_title}」\n\n"
    
    input_text += "評価対象クラスタに所属するテキストデータのサンプル:\n"
    for sample in cluster_samples:
        input_text += f"- {sample}\n"
    
    input_text += "\n他のクラスタからサンプリングしたテキストデータ:\n"
    for sample in other_clusters_samples:
        input_text += f"- {sample}\n"
    
    # LLMに評価リクエスト
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text},
    ]
    
    try:
        response = request_to_chat_openai(messages=messages, model=model, is_json=True)
        evaluation = json.loads(response)
        return evaluation
    except Exception as e:
        print(f"評価中にエラーが発生しました: {e}")
        return {
            "error": str(e),
            "title_evaluation": {
                "inclusiveness_score": {"score": 0, "reason": "評価エラー"},
                "specificity_score": {"score": 0, "reason": "評価エラー"},
                "concreteness_score": {"score": 0, "reason": "評価エラー"},
                "overall_score": {"score": 0, "reason": "評価エラー"},
            }
        }


def evaluate(
    prompt: str,
    clusters_df: pd.DataFrame,
    cluster_column: str,
    text_column: str,
    sample_size: int,
    other_sample_size: int,
    other_sample_clusters: int | None = None,
    max_workers: int | None = None,  # 並列数を指定（Noneなら自動）
) -> list[dict[str, dict]]:
    cluster_ids = sorted(clusters_df[cluster_column].unique())
    evaluation_results = []

    def eval_one(cluster_id):
        cluster_title = clusters_df[clusters_df[cluster_column] == cluster_id]["cluster-level-2-label"].values[0]
        cluster_samples = sample_cluster_data(clusters_df, cluster_column, cluster_id, text_column, sample_size)
        other_clusters_samples = sample_other_clusters_data(
            clusters_df, cluster_column, cluster_id, text_column, other_sample_size, other_sample_clusters
        )
        return {
            "id": cluster_id,
            "label": cluster_title,
            **evaluate_cluster_title(prompt, cluster_title, cluster_samples, other_clusters_samples)
        }

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(eval_one, cluster_id): cluster_id for cluster_id in cluster_ids}
        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            evaluation_results.append(result)

    return evaluation_results

In [53]:
cluster_column = "cluster-level-1-id"
cluster_ids = sorted(clusters_df[cluster_column].unique())


In [54]:
eval_results = evaluate(
    prompt=EVALUATION_PROMPT,
    clusters_df=clusters_df,
    cluster_column="cluster-level-1-id",
    text_column="argument",
    sample_size=30,
    other_sample_size=4,
    other_sample_clusters=10
)

100%|██████████| 15/15 [00:11<00:00,  1.32it/s]


In [55]:
def parse_val(val: dict | str):
    if isinstance(val, dict):
        return val["score"]
    return val

In [56]:
eval_score_df = pd.DataFrame([
        {
            key: parse_val(val_dict)
            for key, val_dict in eval_result.items()} 
            for eval_result in eval_results
            
    ]
)

In [57]:
eval_score_df.sort_values(by="id")

,id,label,inclusiveness_score,specificity_score,concreteness_score
3,1_1,石破政権の不安定性と短命化に関する懸念と憶測,5,5,4
4,1_10,選挙結果に対する多様な反応と意見,5,4,3
2,1_11,選挙結果に関する不確定情報と虚偽の可能性,5,4,4
10,1_12,選挙と野球中継の同時放送に対する視聴者の多様な反応,3,2,3
1,1_13,低投票率に対する懸念とその影響に関する多角的視点,3,4,3
5,1_14,選挙に対する多様な関心と期待の高まり,5,3,3
6,1_15,選挙結果に対する多様な反応と期待,5,4,3
13,1_2,衆院選における過半数割れの可能性とその影響,5,5,4
7,1_3,自民・公明連立政権の過半数維持に対する不安と影響,5,4,4
11,1_4,選挙における政党略称の混乱と公平性への懸念,3,4,4


In [58]:
eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean(axis=0)

inclusiveness_score    4.4
specificity_score      3.8
concreteness_score     3.6
dtype: float64

In [59]:
print("平均スコア", eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean().mean())

平均スコア 3.933333333333333


In [60]:
eval_score_df.sort_values(by="id").to_csv(exp_output_dir / "プロンプト変更後の自動評価結果.csv", index=False)